[DIY Disease Tracking Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020,2024 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# DIY Disease Tracking Dashboard

This is a template for your DIY Disease Tracking Dashboard, to which you can add the code you developed in the previous notebooks. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the QMUL JupyterHub as well as in the Binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

Broadly speaking, Voila acts by **running all the cells in your notebook** when the dashboard is first loaded; it then hides all code cells and displays all markdown cells and any outputs, including widgets. However, the code is still there in the background and handles any interaction with the widgets. To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

In [21]:
# COVID-19 Dashboard
#This dashboard provides an interactive way to explore COVID-19 cases data.

## Features
#- View daily COVID-19 cases over time.
#- Refresh data using the "Refresh Data" button to fetch the latest information.
#- Select data to display using the dropdown menu.


In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import time
import json

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk

You should include "canned" data in ```.json``` files along with your dashboard. When the dashboard starts, it should load that data and assign it as a dictionary to the ```jsondata``` variable (the code below will be hidden when the dashboard is rendered by Voila).

In [4]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
jsondata={}
with open("COVID-19_cases_casesByDay.json", "rt") as infile:
    jsondata["cases"] = json.load(infile)

## Wrangle the data

The dashboard should contain the logic to wrangle the raw data into a ```DataFrame``` (or more than one, as required) that will be used for plotting. The wrangling code should be put into a function and called on the data from the JSON file (we'll need to call it again on any data downloaded from the API).  In this template, we just pretend we are wrangling ```rawdata``` and instead generate a dataframe with some random data

In [7]:
def wrangle_data(rawdata):
    """
    Parameters:
    - rawdata: data from JSON file or API call.

    Returns:
    - A DataFrame suitable for plotting.
    """
    # Initialize the DataFrame
    df = pd.DataFrame(columns=['Date', 'Cases'])

    # Extract and structure data
    data = []
    for entry in rawdata.get("cases", []):  # Assume "cases" key in rawdata
        date = entry.get("date")
        value = entry.get("metric_value", 0.0)
        data.append({"Date": date, "Cases": value})

    # Create the DataFrame
    df = pd.DataFrame(data)
    df["Date"] = pd.to_datetime(df["Date"])
    df.set_index("Date", inplace=True)

    # Fill missing dates if needed
    start_date = df.index.min()
    end_date = df.index.max()
    full_index = pd.date_range(start=start_date, end=end_date, freq='D')
    df = df.reindex(full_index)
    df.fillna(0.0, inplace=True)

    return df

# Example call to wrangle the data
df = wrangle_data(jsondata)  # Replace jsondata with your loaded JSON data

# Display the first few rows to verify
df


,Cases
2020-01-30,1.0
2020-01-31,0.0
2020-02-01,0.0
2020-02-02,1.0
2020-02-03,18.0
...,...
2024-11-22,144.0
2024-11-23,152.0
2024-11-24,103.0
2024-11-25,109.0


## Download current data

Give your users an option to refresh the dataset - a "refresh" button will do. The button callback should
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting (here, ```df```);
* optionally: force a redraw of the graph and give the user some fredback.

Once you get it to work, you may want to wrap your API call inside an exception handler, so that the user is informed, the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached or data are not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

In this example, clicking on the button below just generates some more random data and refreshes the graph. The button should read *Fetch Data*. If you see anything else, take a deep breath :)

In [8]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the UKHSA API. Return data as a like-for-like replacement for the "canned" data loaded from the JSON file. """
    return {} # return data read from the API

In [17]:
# Placeholder for the global DataFrame
# Define the function to access the API
def access_api():
    """
    Accesses the UKHSA API and retrieves fresh data.
    Returns a like-for-like replacement for the 'canned' data.
    """
    try:
        # API URL
        api_url = "https://api.ukhsa-dashboard.data.gov.uk/themes/infectious_disease/sub_themes/respiratory/topics/COVID-19/geography_types/Nation/geographies/England/metrics/COVID-19_cases_casesByDay"
        
        # Make the API request
        response = requests.get(api_url, params={"page_size": 365})
        response.raise_for_status()  # Raise an HTTPError if response is not 200
        return response.json()  # Return the JSON response
    except requests.exceptions.RequestException as e:
        print(f"Error accessing the API: {e}")
        return {}  # Return an empty dictionary on failure

# API Button Callback
def api_button_callback(button):
    """
    Button callback: fetches data from the API, wrangles it, updates the global DataFrame,
    and refreshes the graph.
    """
    global df
    apidata = access_api()
    
    if apidata:
        # Wrangle the data into a DataFrame
        df = wrangle_data(apidata)
        print("Data refreshed successfully!")
    else:
        print("Failed to fetch data from the API.")
    
    # Simulate graph refresh
    refresh_graph()

    # Provide feedback via the button appearance
    apibutton.icon = "check"
    apibutton.description = "Data Updated"


# Define a placeholder for graph refresh (to be implemented in the graph section)
def refresh_graph():
    """
    Forces a redraw of the graph by toggling the widget value.
    Useful after refreshing the data.
    """
    current = stat_selector.value
    options = list(stat_selector.options)
    other = options[0] if current != options[0] else options[1]
    stat_selector.value = other  # Change to another option
    stat_selector.value = current  # Change back to force redraw
    
# Create the refresh button
apibutton = wdg.Button(
    description="Refresh Data",
    disabled=False,
    button_style="info",  # Change button style (e.g., 'success', 'danger', etc.)
    tooltip="Click to download current data from the UKHSA API",
    icon="download"
)

# Register the callback function with the button
apibutton.on_click(api_button_callback)

# Display the button in the notebook
display(apibutton)

Button(button_style='info', description='Refresh Data', icon='download', style=ButtonStyle(), tooltip='Click t…

## Graphs and Analysis

Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

In [20]:
# Graph Refresh Function
def refresh_graph():
    """
    Forces a redraw of the graph by toggling the widget value.
    Useful after refreshing the data.
    """
    current = stat_selector.value
    options = list(stat_selector.options)
    other = options[0] if current != options[0] else options[1]
    stat_selector.value = other  # Change to another option
    stat_selector.value = current  # Change back to force redraw

# Plotting Function
def plot_covid_data(stat):
    """
    Plots the selected statistic (e.g., cases) from the DataFrame.
    """
    if stat in df.columns:
        df[stat].plot(figsize=(10, 5), legend=True)
        plt.title(f"COVID-19 {stat.capitalize()} Over Time")
        plt.xlabel("Date")
        plt.ylabel("Values")
        plt.grid(True)
        plt.show()
    else:
        print("Selected statistic is not available in the DataFrame.")

# Interactive Widget for Statistic Selection
stat_selector = wdg.Dropdown(
    options=df.columns,  # Populate options dynamically from DataFrame columns
    value=df.columns[0],  # Default to the first column
    description="Statistic:",
    disabled=False,
)

# Graph Interaction
graph = wdg.interactive_output(plot_covid_data, {"stat": stat_selector})

# Display Interactive Components
display(stat_selector, graph)
display(apibutton)

Dropdown(description='Statistic:', options=('Cases',), value='Cases')

Output()

Button(button_style='info', description='Refresh Data', icon='download', style=ButtonStyle(), tooltip='Click t…

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and License** Remember that if you deploy your dashboard as a Binder it will be publicly accessible. Change the copyright notice and take credit for your work! Also acknowledge your sources and the conditions of the license by including this notice: "Based on UK Government [data](https://ukhsa-dashboard.data.gov.uk/) published by the [UK Health Security Agency](https://www.gov.uk/government/organisations/uk-health-security-agency) and on the [DIY Disease Tracking Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) by Fabrizio Smeraldi. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/)."